In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import opendatasets as od

In [2]:
od.download("https://www.kaggle.com/datasets/zsinghrahulk/covertype-forest-cover-types")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

Skipping, found downloaded files in "./covertype-forest-cover-types" (use force=True to force download)
cuda


In [3]:
path = r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/gradient_decent/covertype-forest-cover-types/covertype.csv"
data_df = pd.read_csv(path)
data_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [4]:
data_df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Elevation', 'Aspect',
       'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type1',
       'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6',
       'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15',
       'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
       'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
       'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
       'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
       'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
       'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40', 'Cover_Type'],
 

In [5]:
data_df = data_df.drop(columns=(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']), axis=1)

In [6]:
data_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,258,0,510,221,232,148,6279,1,0,0,...,0,0,0,0,0,0,0,0,0,5
1,212,-6,390,220,235,151,6225,1,0,0,...,0,0,0,0,0,0,0,0,0,5
2,268,65,3180,234,238,135,6121,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3,242,118,3090,238,238,122,6211,1,0,0,...,0,0,0,0,0,0,0,0,0,2
4,153,-1,391,220,234,150,6172,1,0,0,...,0,0,0,0,0,0,0,0,0,5


In [7]:
columns=data_df.columns
print(list(columns))

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']


In [8]:
numerical_cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am']
binary_cols = ['Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']
label=['Cover_Type']



In [9]:
print("\n Checking the NAN values")
print(data_df.isnull().sum())

print("\n Checking the empty strings and unusual values")
print(data_df.isin(["", "NA", "N/A", "NONE", "none", "na", "NULL", "null", "None"]).sum())

print("\n Checking the duplicates")
print(data_df.duplicated().sum())


 Checking the NAN values
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area                       0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0
Soil_Type12                           0
Soil_Type13                           0
Soil_Type14   

In [10]:
data_df.dropna(inplace=True)

In [11]:
scaler = StandardScaler()
data_df[numerical_cols] = scaler.fit_transform(data_df[numerical_cols])
X = data_df.iloc[:,:-1]
y=data_df[label]

In [12]:
data_df.shape

(581012, 52)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)


In [14]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [15]:
X_train.shape, X_test.shape, X_val.shape

((522910, 51), (29051, 51), (29051, 51))

In [16]:
class dataset(Dataset):
    def __init__(self,X ,y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [17]:
training_dataset = dataset(X_train, y_train)
validation_dataset = dataset(X_val, y_val)
testing_dataset = dataset(X_test, y_test)

In [18]:
train_dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
testing_dataloader = DataLoader(testing_dataset, batch_size= 64, shuffle= False)

In [19]:
for x , y in train_dataloader:
    print(x)
    print(y)
    break

tensor([[-1.2676, -0.7963, -0.5606,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.3791, -0.2302,  1.1056,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7884, -1.0364, -0.9352,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.1265, -0.7620,  1.0337,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0472,  2.9262,  0.4520,  ...,  0.0000,  0.0000,  1.0000],
        [-1.2676, -0.7963,  0.3308,  ...,  0.0000,  0.0000,  0.0000]])
tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [2],
        [1],
        [2],
        [2],
        [2],
        [2],
        [2],
        [1],
        [2],
        [2],
        [1],
        [2],
        [7],
        [2],
        [2],
        [1],
        [2],
        [1],
        [2],
        [2],
        [2],
        [3],
        [2],
        [1],
        [1],
        [2],
        [2],
        [1],
        [2],
        [1],
        [2],
        [1],
        [2],
        [2],
        [2],
        [1],
        [6],
        [2],
       

In [20]:
Hidden_Neurons =108
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential (
            nn.Linear(X.shape[1], Hidden_Neurons),
            nn.ReLU(),
            nn.Linear(Hidden_Neurons, 64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU() 
        )
        
        self.classifier= nn.Sequential(
            nn.Linear(16,7),
            
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    

model = MyModel().to(device)
#yo model ko output vanya batch size * 7 (7 ota calsses vako le)

In [21]:
summary(model, input_size=(X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 108]           5,616
              ReLU-2                  [-1, 108]               0
            Linear-3                   [-1, 64]           6,976
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 32]           2,080
              ReLU-6                   [-1, 32]               0
            Linear-7                   [-1, 16]             528
              ReLU-8                   [-1, 16]               0
            Linear-9                    [-1, 7]             119
Total params: 15,319
Trainable params: 15,319
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.06
Estimated Total Size (MB): 0.06
---------------------------------------------

In [22]:
criterion =nn.CrossEntropyLoss()
optimizer =  SGD(model.parameters(), lr = 0.001, momentum=0.9)

In [23]:
epochs =20

In [24]:
for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        
        X = inputs.to(device)
        y = (labels.to(device).squeeze(1)-1).long()
        
        logits = model(X)
        batch_loss = criterion(logits, y)
        batch_loss.backward()
        optimizer.step()
        
        train_loss += batch_loss.item() * y.size(0)
        _, preds = torch.max(logits, dim=1)
        train_acc += (preds == y).sum().item()
    
    avg_train_loss = train_loss / len(training_dataset)
    avg_train_acc = train_acc / len(training_dataset)
    
    # Validation
    val_loss = 0
    val_acc = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            X = inputs.to(device)
            y = (labels.to(device).squeeze(1)-1).long()
            
            logits = model(X)
            batch_loss = criterion(logits, y)
            
            val_loss += batch_loss.item() * y.size(0)
            _, preds = torch.max(logits, dim=1)
            val_acc += (preds == y).sum().item()
    
    avg_val_loss = val_loss / len(validation_dataset)
    avg_val_acc = val_acc / len(validation_dataset)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f" Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")
    print(f" Val Loss:   {avg_val_loss:.4f}, Val Acc:   {avg_val_acc:.4f}")

        
        

    


Epoch 1/20
 Train Loss: 1.0376, Train Acc: 0.5348
 Val Loss:   0.8325, Val Acc:   0.6437
Epoch 2/20
 Train Loss: 0.7559, Train Acc: 0.6691
 Val Loss:   0.7019, Val Acc:   0.6947
Epoch 3/20
 Train Loss: 0.6824, Train Acc: 0.7039
 Val Loss:   0.6500, Val Acc:   0.7224
Epoch 4/20
 Train Loss: 0.6423, Train Acc: 0.7214
 Val Loss:   0.6162, Val Acc:   0.7338
Epoch 5/20
 Train Loss: 0.6132, Train Acc: 0.7324
 Val Loss:   0.5978, Val Acc:   0.7364
Epoch 6/20
 Train Loss: 0.5885, Train Acc: 0.7440
 Val Loss:   0.5898, Val Acc:   0.7478
Epoch 7/20
 Train Loss: 0.5692, Train Acc: 0.7524
 Val Loss:   0.5567, Val Acc:   0.7604
Epoch 8/20
 Train Loss: 0.5537, Train Acc: 0.7591
 Val Loss:   0.5456, Val Acc:   0.7636
Epoch 9/20
 Train Loss: 0.5397, Train Acc: 0.7661
 Val Loss:   0.5314, Val Acc:   0.7718
Epoch 10/20
 Train Loss: 0.5273, Train Acc: 0.7711
 Val Loss:   0.5263, Val Acc:   0.7738
Epoch 11/20
 Train Loss: 0.5162, Train Acc: 0.7763
 Val Loss:   0.5171, Val Acc:   0.7780
Epoch 12/20
 Train 

In [25]:
def test_model(model, test_dataloader, criterion, device):
    model.eval()
    test_loss = 0
    test_acc = 0
    
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            X = inputs.to(device)
            y = (labels.to(device).squeeze(1) - 1).long()
            
            logits = model(X)
            batch_loss = criterion(logits, y)
            
            test_loss += batch_loss.item() * y.size(0)
            _, preds = torch.max(logits, dim=1)
            test_acc += (preds == y).sum().item()
    
    avg_test_loss = test_loss / len(test_dataloader.dataset)
    avg_test_acc = test_acc / len(test_dataloader.dataset)
    
    print(f"Test Loss: {avg_test_loss:.4f}, Test Acc: {avg_test_acc:.4f}")
    
    return avg_test_loss, avg_test_acc

# Example usage after training:
test_loss, test_acc = test_model(model, testing_dataloader, criterion, device)


Test Loss: 0.4395, Test Acc: 0.8106


Adam Optimizer which also covers RMSPROP

In [21]:
Hidden_Neurons = 108

class MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(X.shape[1], Hidden_Neurons),
            nn.ReLU(),
            nn.Linear(Hidden_Neurons, 64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(16,7)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    

mymodel = MODEL().to(device)

In [22]:
summary(mymodel, input_size= (X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 108]           5,616
              ReLU-2                  [-1, 108]               0
            Linear-3                   [-1, 64]           6,976
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 32]           2,080
              ReLU-6                   [-1, 32]               0
            Linear-7                   [-1, 16]             528
              ReLU-8                   [-1, 16]               0
            Linear-9                    [-1, 7]             119
Total params: 15,319
Trainable params: 15,319
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.06
Estimated Total Size (MB): 0.06
---------------------------------------------

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params= mymodel.parameters(), lr=0.001, betas=(0.9, 0.999), eps = 1e-8)

In [38]:
epochs = 20
def training_validation(model,train_dataloader, validation_dataloader, criterion, optimizer):
    for epoch in range(epochs):
        total_train_loss = 0
        total_train_acc = 0
       
        
        for data, label in train_dataloader:
            optimizer.zero_grad()
            X = data.to(device)
            y = (label.to(device).squeeze(1) - 1).long()
            logits = model(X)
            batch_loss = criterion(logits,y)
            batch_loss.backward()
            optimizer.step()
            
            total_train_loss += batch_loss.item() * y.size(0)
            _, preds = torch.max(logits, dim=1)
            total_train_acc += (preds == y).sum().item()
            
        avg_train_loss = total_train_loss/len(training_dataset)
        avg_train_acc = total_train_acc/len(training_dataset)
    

        # Validation
        val_loss = 0
        val_acc = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in validation_dataloader:
                X = inputs.to(device)
                y = (labels.to(device).squeeze(1)-1).long()
                
                logits = model(X)
                batch_loss = criterion(logits, y)
                
                val_loss += batch_loss.item() * y.size(0)
                _, preds = torch.max(logits, dim=1)
                val_acc += (preds == y).sum().item()
        
        avg_val_loss = val_loss / len(validation_dataset)
        avg_val_acc = val_acc / len(validation_dataset)
    
        print(f"Epoch {epoch+1}/{epochs}")
        print(f" Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")
        print(f" Val Loss:   {avg_val_loss:.4f}, Val Acc:   {avg_val_acc:.4f}")

training_validation(mymodel, train_dataloader, validation_dataloader, criterion, optimizer)

Epoch 1/20
 Train Loss: 0.4851, Train Acc: 0.7850
 Val Loss:   0.4769, Val Acc:   0.7906
Epoch 2/20
 Train Loss: 0.4655, Train Acc: 0.7940
 Val Loss:   0.4639, Val Acc:   0.7956
Epoch 3/20
 Train Loss: 0.4493, Train Acc: 0.8017
 Val Loss:   0.4572, Val Acc:   0.7966
Epoch 4/20
 Train Loss: 0.4361, Train Acc: 0.8080
 Val Loss:   0.4456, Val Acc:   0.8021
Epoch 5/20
 Train Loss: 0.4242, Train Acc: 0.8146
 Val Loss:   0.4297, Val Acc:   0.8109
Epoch 6/20
 Train Loss: 0.4138, Train Acc: 0.8195
 Val Loss:   0.4219, Val Acc:   0.8151
Epoch 7/20
 Train Loss: 0.4049, Train Acc: 0.8235
 Val Loss:   0.4155, Val Acc:   0.8205
Epoch 8/20
 Train Loss: 0.3979, Train Acc: 0.8270
 Val Loss:   0.4002, Val Acc:   0.8291
Epoch 9/20
 Train Loss: 0.3914, Train Acc: 0.8302
 Val Loss:   0.3992, Val Acc:   0.8264
Epoch 10/20
 Train Loss: 0.3852, Train Acc: 0.8329
 Val Loss:   0.3953, Val Acc:   0.8289
Epoch 11/20
 Train Loss: 0.3796, Train Acc: 0.8358
 Val Loss:   0.3987, Val Acc:   0.8285
Epoch 12/20
 Train 

In [39]:
def test_model(model, test_dataloader, criterion, device):
    model.eval()
    test_loss = 0
    test_acc = 0
    
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            X = inputs.to(device)
            y = (labels.to(device).squeeze(1) - 1).long()
            
            logits = model(X)
            batch_loss = criterion(logits, y)
            
            test_loss += batch_loss.item() * y.size(0)
            _, preds = torch.max(logits, dim=1)
            test_acc += (preds == y).sum().item()
    
    avg_test_loss = test_loss / len(test_dataloader.dataset)
    avg_test_acc = test_acc / len(test_dataloader.dataset)
    
    print(f"Test Loss: {avg_test_loss:.4f}, Test Acc: {avg_test_acc:.4f}")
    
    return avg_test_loss, avg_test_acc

# Example usage after training:
test_loss, test_acc = test_model(mymodel, testing_dataloader, criterion, device)


Test Loss: 0.3559, Test Acc: 0.8475
